# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weathercity_df = "../output/weather_cities.csv"
part1_df = pd.read_csv(weathercity_df)
part1_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,avarua,75,CK,1619849483,83,-21.2078,-159.7750,77.00,4.61
1,nikolskoye,0,RU,1619849483,65,59.7035,30.7861,42.01,6.71
2,norman wells,20,CA,1619849483,52,65.2820,-126.8329,37.40,5.75
3,faanui,91,PF,1619849216,69,-16.4833,-151.7500,81.45,7.65
4,khatanga,100,RU,1619849484,97,71.9667,102.5000,33.30,9.15
...,...,...,...,...,...,...,...,...,...
563,tongren,89,CN,1619849652,44,27.7172,109.1853,89.53,4.43
564,vidor,90,US,1619849652,94,30.1316,-94.0155,69.80,5.75
565,lethem,19,GY,1619849653,86,3.3803,-59.7968,75.13,7.31
566,kemi,0,FI,1619849653,47,65.8596,24.8228,30.20,10.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Create a map using coordinates 
marker_locations = part1_df[['Lat', 'Lng']]
weights = part1_df['Humidity'].astype(float)
# Plot Heatmap
fig = gmaps.figure(zoom_level = 1.8, center = [10,50])

# Create heat layer
fig.add_layer(gmaps.heatmap_layer(marker_locations,  weights= weights,
                                 dissipating=False, max_intensity=100,
                                 point_radius=2.2))

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Fitting my weather criteria:
high_temp = 80
low_temp = 70
new_df = part1_df.loc[(part1_df['Max Temp']< high_temp) & (part1_df['Max Temp'] > low_temp)        
                                                        & (part1_df['Cloudiness'] <= 0)
                                                        & (part1_df['Wind Speed'] < 24)
                                                        & (part1_df['Humidity'] > 35)
                                                        & (part1_df['Humidity'] < 70),:]        
new_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
67,hobart,0,AU,1619849502,52,-42.8794,147.3294,71.01,6.91
272,namibe,0,AO,1619849355,68,-15.1961,12.1522,76.12,4.16
312,lakkion,0,GR,1619849575,43,37.1333,26.8500,73.40,3.44
399,murgab,0,TM,1619849599,56,37.4966,61.9714,73.40,6.91
469,noumea,0,NC,1619849622,69,-22.2763,166.4572,75.20,13.80
472,adana,0,TR,1619849623,68,36.9850,35.2881,75.20,2.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(new_df)
hotel_df['Hotel Name'] = ''
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
67,hobart,0,AU,1619849502,52,-42.8794,147.3294,71.01,6.91,
272,namibe,0,AO,1619849355,68,-15.1961,12.1522,76.12,4.16,
312,lakkion,0,GR,1619849575,43,37.1333,26.8500,73.40,3.44,
399,murgab,0,TM,1619849599,56,37.4966,61.9714,73.40,6.91,
469,noumea,0,NC,1619849622,69,-22.2763,166.4572,75.20,13.80,
472,adana,0,TR,1619849623,68,36.9850,35.2881,75.20,2.30,


In [7]:
from pprint import pprint 
# find the closest hotel of each type to coordinates
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#setting parameters
params = {
        "type": "hotel",
        "radius": 5000,
        "key": g_key
         }
# Use the lat/lng recovered to identify hotels
for index, row in hotel_df.iterrows():
    # Get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change locations
    params["location"] = f"{lat},{lng}"
    
    # Make API request and convert to json
    response_hotel = requests.get(base_url, params=params).json()
    
    # Print the json (pretty printed) 
    # pprint(response_hotel)
    results = response_hotel["results"]
    # Try to grab the hotel name if is available in the API
    try:
        print(f"The first Hotel result for location : {lat},{lng} is: Hotel {results[0]['name']}.")
        #Storing the first Hotel result into the DataFrame
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name'] 
    # Handle exceptions
    except KeyError as e:
        message ='Key Error: ' + str(e)
        print(message)
        print("Skipping...")
    except IndexError as e:
        message ='Key Error: ' + str(e)
        print(message)
        print("Skipping...")
print("------------")

The first Hotel result for location : -42.8794,147.3294 is: Hotel Hobart.
The first Hotel result for location : -15.1961,12.1522 is: Hotel Namibe.
The first Hotel result for location : 37.1333,26.85 is: Hotel Agia Marina.
The first Hotel result for location : 37.4966,61.9714 is: Hotel Mary.
The first Hotel result for location : -22.2763,166.4572 is: Hotel Nouméa.
The first Hotel result for location : 36.985,35.2881 is: Hotel Adana.
------------


In [8]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
67,hobart,0,AU,1619849502,52,-42.8794,147.3294,71.01,6.91,Hobart
272,namibe,0,AO,1619849355,68,-15.1961,12.1522,76.12,4.16,Namibe
312,lakkion,0,GR,1619849575,43,37.1333,26.8500,73.40,3.44,Agia Marina
399,murgab,0,TM,1619849599,56,37.4966,61.9714,73.40,6.91,Mary
469,noumea,0,NC,1619849622,69,-22.2763,166.4572,75.20,13.80,Nouméa
472,adana,0,TR,1619849623,68,36.9850,35.2881,75.20,2.30,Adana


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content= hotel_info)
# Add the layer to the map
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

#### Screenshot of Heat Map done by me
![map.png](Images/map.png)